In [9]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import numpy as np

class CustomFashionMNIST(Dataset):
    def __init__(self, root='./data', train=True, transform=None):
        self.original_dataset = datasets.FashionMNIST(
            root=root,
            train=train,
            download=True
        )
        
        self.data = self.original_dataset.data.numpy()
        self.targets = self.original_dataset.targets.numpy()
        self.data = self.data.astype(np.float32) / 255.0
        self.transform = transform
        
        self.classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                       'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.targets[idx]
        image = torch.FloatTensor(image).unsqueeze(0)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def save_model(model, optimizer, epoch, accuracy, is_best=False):
    """Save model weights - both best and last"""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'accuracy': accuracy
    }
    
    # Always save the last model
    last_filename = 'fashion_model_last.pt'
    torch.save(checkpoint, last_filename)  # Removed weights_only parameter
    print(f"Last model saved to {last_filename}")
    
    # Save best model separately if it's the best
    if is_best:
        best_filename = 'fashion_model_best.pt'
        torch.save(checkpoint, best_filename)  # Removed weights_only parameter
        print(f"Best model saved to {best_filename}")

def load_model(filename='fashion_model_best.pt'):
    """Load model weights"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = FashionNet().to(device)
    
    try:
        checkpoint = torch.load(filename, map_location=device)  # Removed weights_only parameter
        model.load_state_dict(checkpoint['model_state_dict'])
        accuracy = checkpoint.get('accuracy', 0.0)
        epoch = checkpoint.get('epoch', 0)
        print(f"Model loaded from {filename}")
        print(f"Accuracy: {accuracy:.2f}%")
        return model, accuracy, epoch
    except FileNotFoundError:
        print(f"No saved model found at {filename}")
        return model, 0.0, 0

if __name__ == "__main__":
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Create datasets and loaders
    train_dataset = CustomFashionMNIST(train=True)
    test_dataset = CustomFashionMNIST(train=False)
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    # Create model
    model = FashionNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    # Train for 100 epochs
    num_epochs = 100
    best_accuracy = 0.0
    
    print("Starting training...")
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')
        
        # Evaluation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Epoch: {epoch}, Accuracy: {accuracy:.2f}%')
        
        # Save both best and last models
        is_best = accuracy > best_accuracy
        if is_best:
            best_accuracy = accuracy
            print(f'New best accuracy: {accuracy:.2f}%')
        save_model(model, optimizer, epoch, accuracy, is_best=is_best)
        
    print(f"\nTraining completed!")
    print(f"Best accuracy achieved: {best_accuracy:.2f}%")

Using device: cuda
Starting training...
Epoch: 0, Batch: 0, Loss: 2.2376
Epoch: 0, Batch: 100, Loss: 0.3818
Epoch: 0, Batch: 200, Loss: 0.3152
Epoch: 0, Batch: 300, Loss: 0.4729
Epoch: 0, Batch: 400, Loss: 0.3646
Epoch: 0, Batch: 500, Loss: 0.3664
Epoch: 0, Batch: 600, Loss: 0.4031
Epoch: 0, Batch: 700, Loss: 0.2394
Epoch: 0, Batch: 800, Loss: 0.3621
Epoch: 0, Batch: 900, Loss: 0.2667
Epoch: 0, Accuracy: 89.15%
New best accuracy: 89.15%
Last model saved to fashion_model_last.pt
Best model saved to fashion_model_best.pt
Epoch: 1, Batch: 0, Loss: 0.3146
Epoch: 1, Batch: 100, Loss: 0.3488
Epoch: 1, Batch: 200, Loss: 0.2328
Epoch: 1, Batch: 300, Loss: 0.1447
Epoch: 1, Batch: 400, Loss: 0.2826
Epoch: 1, Batch: 500, Loss: 0.5003
Epoch: 1, Batch: 600, Loss: 0.2192
Epoch: 1, Batch: 700, Loss: 0.1918
Epoch: 1, Batch: 800, Loss: 0.1484
Epoch: 1, Batch: 900, Loss: 0.1819
Epoch: 1, Accuracy: 90.90%
New best accuracy: 90.90%
Last model saved to fashion_model_last.pt
Best model saved to fashion_mode

In [13]:
import torch
from torch.utils.data import DataLoader


def evaluate_model(model_path='fashion_model_best.pt'):  # Fixed function definition
    """Evaluate a saved model"""
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    try:
        # Load the saved model
        model, saved_accuracy, last_epoch = load_model(model_path)  # Now model_path is defined
        model.eval()

        # Create test dataset
        test_dataset = CustomFashionMNIST(train=False)
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # Evaluation
        correct = 0
        total = 0

        print(f"\nEvaluating model from epoch {last_epoch}")
        print(f"Saved accuracy: {saved_accuracy:.2f}%")

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        current_accuracy = 100 * correct / total
        print(f'Current Evaluation Accuracy: {current_accuracy:.2f}%')

        # Show sample predictions
        print("Sample Predictions:")
        num_samples = 5  # Number of sample predictions to show
        for i in range(num_samples):
            sample_image, sample_label = test_dataset[i]
            sample_image = sample_image.unsqueeze(0).to(device)
            prediction = model(sample_image).argmax(1)[0].item()
            print(f"Sample {i + 1}:")
            print(f"Model predicts: {test_dataset.classes[prediction]}")
            print(f"Actual label: {test_dataset.classes[sample_label]}")
            print()
            
    except FileNotFoundError:
        print(f"Error: Could not find model file '{model_path}'")
    except Exception as e:
        print(f"Error during evaluation: {str(e)}")

if __name__ == "__main__":    # Fixed the name dunder
    print(" Fashion MNIST Model Evaluation")
    
    print("Evaluating Best Model:")
    evaluate_model('fashion_model_best.pt')
    
    print("Evaluating Last Model:")
    evaluate_model('fashion_model_last.pt')

 Fashion MNIST Model Evaluation
Evaluating Best Model:
Using device: cuda
Model loaded from fashion_model_best.pt
Accuracy: 93.12%

Evaluating model from epoch 97
Saved accuracy: 93.12%


/tmp/ipykernel_3185871/1962039490.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, map_location=device)  # Removed weights_only paramet

Current Evaluation Accuracy: 93.12%
Sample Predictions:
Sample 1:
Model predicts: Ankle boot
Actual label: Ankle boot

Sample 2:
Model predicts: Pullover
Actual label: Pullover

Sample 3:
Model predicts: Trouser
Actual label: Trouser

Sample 4:
Model predicts: Trouser
Actual label: Trouser

Sample 5:
Model predicts: Shirt
Actual label: Shirt

Evaluating Last Model:
Using device: cuda
Model loaded from fashion_model_last.pt
Accuracy: 92.47%

Evaluating model from epoch 99
Saved accuracy: 92.47%
Current Evaluation Accuracy: 92.47%
Sample Predictions:
Sample 1:
Model predicts: Ankle boot
Actual label: Ankle boot

Sample 2:
Model predicts: Pullover
Actual label: Pullover

Sample 3:
Model predicts: Trouser
Actual label: Trouser

Sample 4:
Model predicts: Trouser
Actual label: Trouser

Sample 5:
Model predicts: Shirt
Actual label: Shirt

